##  概要

本案例演示的是将点云从一个参考坐标系转换为另外一个参考坐标系。 

```
@作者: 阿凯爱玩机器人
@QQ: 244561792
@微信: xingshunkai
@邮箱: xingshunkai@qq.com
@网址: deepsenserobot.com
@B站: "阿凯爱玩机器人"
```

## 导入依赖

In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import open3d as o3d

# 自定义库
from kyle_robot_toolbox.transform import Transform
from kyle_robot_toolbox.open3d import *
from kyle_robot_toolbox.camera import Gemini335

np.set_printoptions(precision=3, suppress=True)

pybullet build time: Jun  3 2022 02:05:55


## 创建相机对象

In [2]:
# 创建相机对象
camera = Gemini335()

## 载入数据

In [3]:
# 从文件中载入彩图
# 从文件中载入深度图
img_name = "radon" # 图像ID
color_image = cv2.imread(f"./data/example/radon_caliboard_pose_rgb/{img_name}.png")
# 保存深度图 二进制格式
depth_image = np.load(f"./data/example/radon_caliboard_pose_rgb/{img_name}.npy")

In [4]:
# PCD点云
pcd = camera.get_pcd(color_image, depth_image, camera="rgb_camera")
# 绘制相机坐标系与点云
# 创建坐标系的Mesh
corrd_mesh_cam = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)

draw_geometry([pcd, corrd_mesh_cam], bk_color=[0.2, 0.5, 8.0])

## 点云空间变换

将PCD点云从相机坐标系转换为标定板坐标系。
转换公式如下:

$$
^{board}P = ^{board}_{cam}T * ^{cam}P
$$

In [5]:
# 标定板坐标系在相机坐标系下的位姿
T_cam2board = np.loadtxt("./data/example/radon_caliboard_pose_rgb/T_cam2board.txt",\
                    delimiter=",")

In [6]:
# 求解标定板坐标系到相机坐标系的空间变换
T_board2cam = Transform.inverse(T_cam2board)
print("标定板坐标系到相机坐标系的空间变换")
print(T_board2cam)

标定板坐标系到相机坐标系的空间变换
[[-0.999  0.032 -0.012 -0.007]
 [ 0.032  0.999 -0.002 -0.009]
 [ 0.012 -0.002 -1.     0.314]
 [ 0.     0.     0.     1.   ]]


In [7]:
# 点云空间变换
pcd_in_board = copy.deepcopy(pcd)
pcd_in_board.transform(T_board2cam)

# 绘制标定板坐标系与点云
# 创建坐标系的Mesh
corrd_mesh_board = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)

draw_geometry([pcd_in_board, corrd_mesh_board], bk_color=[0.2, 0.5, 8.0])